# Upload your image

* In the top left-hand corner click "File"
* Click "Open..." from the drop down menu
* Click the folder named "images"
* In the top right-hand corner click "Upload"
* Choose your image from your computer 
* Click the blue "Upload" button 

# How to read code

* Everything after a hashtag (light green and italic) is a comment. The computer doesn't read this; these are notes for the humans reading the code!
* `pcv.`____ are functions from the PlantCV software library. These functions DO something (i.e. read in an image, resize the image, ...)
* Stuff inside parentheses are parameters. These are the ingredients where your function is the recipe.
* Stuff to the left of equals sign are the things "returned" by a function. Generally, a function will spit out one or more objects that get saved to variable names. These output objects are usually the input for the next step. 

# Update your code to read your new image 

* In the block of code that starts `class options:` change
the line where `self.image` gets assigned

In [ ]:
# Import software needed 
from plantcv import plantcv as pcv 
import numpy as np

In [ ]:
class options:
    def __init__(self):
        self.image = "images/maize.JPG"
        #                    ^
        #                    |
        # Replace "maize.JPG" with your image name. 
        # NOTE: this is case sensitive! 
        self.debug = "plot"
        self.writeimg= False 
        self.result = "./g2p_results"
        self.outdir = "."
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug


In [ ]:
# Read image (sometimes you need to run this line twice to see the image) 

# Inputs:
#   filename - Image file to be read in 
#   mode - Return mode of image; either 'native' (default), 'rgb', 'gray', or 'csv' 
img, path, filename = pcv.readimage(filename=args.image)


In [ ]:
# The image is quite large which can slow down computation, so resize

# Inputs:
#   img - RGB or grayscale image
#   resize_x - How much to resize in the x axis
#   resize_y - How much to resize in the y axis
img = pcv.resize(img=img, resize_x=.5, resize_y=.5)


In [ ]:
# Convert RGB to LAB and extract the green-magenta channel ('a')

# Input:
#   rgb_img - RGB image data 
#   channel- Split by 'l' (lightness), 'a' (green-magenta), or 'b' (blue-yellow) channel
a_img = pcv.rgb2gray_lab(rgb_img=img, channel='a')


In [ ]:
# Threshold can be on either light or dark objects in the image. 

# Inputs:
#   gray_img - Grayscale image data 
#   threshold- Threshold value (between 0-255)
#   max_value - Value to apply above threshold (255 = white) 
#   object_type - 'light' (default) or 'dark'. If the object is lighter than the background then standard threshold is done.
#                 If the object is darker than the background then inverse thresholding is done. 
a_thresh_img = pcv.threshold.binary(gray_img=a_img, threshold=125, max_value=255, object_type='dark')


In [ ]:
# Filter out dark noise from an image.

# Inputs:
#   gray_img - Grayscale or binary image data
#   kernel - Optional neighborhood, expressed as an array of 1's and 0's. If None (default),
#   uses cross-shaped structuring element.
closed = pcv.closing(gray_img=a_thresh_img)


In [ ]:
# Fill small objects (reduce image noise) 

# Inputs: 
#   bin_img - Binary image data 
#   size - Minimum object area size in pixels (must be an integer), and smaller objects will be filled
filled = pcv.fill(bin_img=closed, size=40)


In [ ]:
# Dilate the mask to avoid losing leaf tips

# Inputs:
#    gray_img = input image
#    ksize    = kernel size, integer
#    i        = iterations, i.e. number of consecutive filtering passes
dilated = pcv.dilate(gray_img=filled, ksize=5, i=1)


In [ ]:
# Use a lowpass (blurring) filter to smooth sobel image

# Inputs:
#   gray_img - Grayscale image data 
#   ksize - Kernel size (integer or tuple), (ksize, ksize) box if integer input,
#           (n, m) box if tuple input 
m_blur = pcv.median_blur(gray_img=dilated, ksize=12)



In [ ]:
# Fill in any holes in the plant mask

# Inputs:
#   bin_img - Binary image data
filled_mask = pcv.fill_holes(bin_img=m_blur)

In [ ]:
# Identify objects

# Inputs: 
#   img - RGB or grayscale image data for plotting 
#   mask - Binary mask used for detecting contours 
obj_cnt, obj_hierarchy = pcv.find_objects(img=img, mask=filled_mask)

In [ ]:
# Define region of interest (ROI)

# Inputs: 
#   img - RGB or grayscale image to plot the ROI on 
#   x - The x-coordinate of the upper left corner of the rectangle 
#   y - The y-coordinate of the upper left corner of the rectangle 
#   h - The height of the rectangle 
#   w - The width of the rectangle 
roi_cnt, roi_hierarchy = pcv.roi.rectangle(img=img, x=500, y=500, h=1000, w=2000)


In [ ]:
# Decide which objects to keep

# Inputs:
#    img            = img to display kept objects
#    roi_contour    = contour of roi, output from any ROI function
#    roi_hierarchy  = contour of roi, output from any ROI function
#    object_contour = contours of objects, output from pcv.find_objects function
#    obj_hierarchy  = hierarchy of objects, output from pcv.find_objects function
#    roi_type       = 'partial' (default, for partially inside), 'cutto', or 
#    'largest' (keep only largest contour)
plant_obj, plant_hier, plant_mask, obj_area = pcv.roi_objects(img=img, roi_contour=roi_cnt, 
                                                               roi_hierarchy=roi_hierarchy, 
                                                               object_contour=obj_cnt, 
                                                               obj_hierarchy=obj_hierarchy,
                                                               roi_type='partial')


In [ ]:
# Skeletonize the plant mask (one-pixel wide representation)

# Inputs:
#   mask - Binary mask 
skeleton = pcv.morphology.skeletonize(mask=plant_mask)

In [ ]:
# Adjust line thickness with the global line thickness parameter (default = 5),
# and provide binary mask of the plant for debugging. NOTE: the objects and
# hierarchies returned will be exactly the same but the debugging image (segmented_img)
# will look different.
pcv.params.line_thickness = 10

# Prune the skeleton  

# Inputs:
#   skel_img = Skeletonized image
#   size     = Pieces of skeleton smaller than `size` should get removed. (Optional) Default `size=0`. 
#   mask     = Binary mask for debugging (optional). If provided, debug images will be overlaid on the mask.

pruned, seg_img, edge_objects = pcv.morphology.prune(skel_img=skeleton, size=100, mask=plant_mask)


In [ ]:
# Sort segments into leaf objects and stem objects  

# Inputs:
#   skel_img  = Skeletonized image
#   objects   = List of contours
#   mask      = (Optional) binary mask for debugging. If provided, debug image 
#                will be overlaid on the mask.

leaf_obj, stem_obj = pcv.morphology.segment_sort(skel_img=pruned, 
                                                 objects=edge_objects,
                                                 mask=plant_mask)


In [ ]:
# Similar to line thickness, there are optional text size and text thickness parameters 
# that can be adjusted to better suit images or varying sizes.
pcv.params.text_size=3 # (default text_size=.55)
pcv.params.text_thickness=8 # (defaul text_thickness=2) 

# Identify segments     

# Inputs:
#   skel_img  = Skeletonized image
#   objects   = List of contours
#   mask      = (Optional) binary mask for debugging. If provided, debug image 
#                will be overlaid on the mask.

segmented_img, labeled_img = pcv.morphology.segment_id(skel_img=skeleton,
                                                       objects=leaf_obj,
                                                       mask=plant_mask)


In [ ]:
# Measure path lengths of segments     

# Inputs:
#   segmented_img = Segmented image to plot lengths on
#   objects       = List of contours

labeled_img  = pcv.morphology.segment_path_length(segmented_img=segmented_img, 
                                                  objects=leaf_obj)


In [ ]:
# Measure euclidean distance of segments      

# Inputs:
#   segmented_img = Segmented image to plot lengths on
#   objects       = List of contours

labeled_img = pcv.morphology.segment_euclidean_length(segmented_img=segmented_img, 
                                                      objects=leaf_obj)


In [ ]:
# Measure the angle of segments      

# Inputs:
#   segmented_img = Segmented image to plot angles on
#   objects       = List of contours

labeled_img = pcv.morphology.segment_angle(segmented_img=segmented_img, 
                                           objects=leaf_obj)


In [ ]:
# Measure the leaf insertion angles   
# NOTE: This function is slow and will likely take up to 2 minutes to run

# Inputs:
#   skel_img         = Skeletonize image 
#   segmented_img    = Segmented image to plot insertion angles on
#   leaf_objects     = List of leaf contours
#   stem_objects     = List of stem objects 
#   size             = Size of the inner portion of each leaf to find a linear regression line

labeled_img = pcv.morphology.segment_insertion_angle(skel_img=skeleton,
                                                     segmented_img=segmented_img, 
                                                     leaf_objects=leaf_obj, 
                                                     stem_objects=stem_obj,
                                                     size=90)


In [ ]:
# Format data collected into a table 

leaf_ids = np.vstack(pcv.outputs.observations['segment_path_length']['label'])

segment_path_length = np.vstack(pcv.outputs.observations['segment_path_length']['value'])

segment_eu_length = np.vstack(pcv.outputs.observations['segment_eu_length']['value'])

seg_angles = np.vstack(pcv.outputs.observations['segment_angle']['value'])

segment_insertion_angle = np.vstack(pcv.outputs.observations['segment_insertion_angle']['value'])

data_table = np.hstack((leaf_ids, segment_path_length, segment_eu_length, seg_angles, segment_insertion_angle))


In [ ]:
# Print data out to a text file that can be imported into Excel 

np.savetxt("leaf_phenotype_data.txt", data_table, delimiter=',', fmt='%10.5f',
          header='leaf_id, path_length, eu_length, angle, insertion_angle')

# To see the text file saved out, click 'File' tab in top left corner, click 'Open'

# Download this file to your computer by checking the box directly to the 
# left of the file named "leaf_phenotype_data.txt" and then click "Download" 
# in the top left corner. 

